# Making slides with the Assistants API and DALL-E3

This notebook illustrates the use of the new [**Assistants API**](https://platform.openai.com/docs/assistants/overview) and [**DALL-E3**](https://platform.openai.com/docs/guides/images?context=node) in crafting informative and visually appealing slides. <br>
Creating engaging slides is a pivotal aspect of many jobs, but it can be a laborious and time-consuming task. Additionally, extracting insights from data and articulating them effectively on slides can be challenging. <br><br> This cookbook recipe will demonstrate how you can utilize the new Assistants API, with GPT-Vision, and DALL-E3 to create slides for you without you having to touch Microsoft PowerPoint or Google Slides, saving you valuable time and effort!

## 0. Setup

In [1]:
from IPython.display import display, Image
from openai import OpenAI
import pandas as pd
import json
import io
from PIL import Image
import requests


client = OpenAI()

#Lets import some helper functions for assistants from https://cookbook.openai.com/examples/assistants_api_overview_python
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

def submit_message(assistant_id, thread, user_message,file_ids=None):
    params = {
        'thread_id': thread.id,
        'role': 'user',
        'content': user_message,
    }
    if file_ids:
        params['file_ids']=file_ids

    client.beta.threads.messages.create(
        **params
)
    return client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)

def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id)


## 1. Creating the content

In this recipe, we will be creating a brief fictional presentation for the quarterly financial review of our company, NotReal Corporation. In the presentation, we want to highlight some key trends we are seeing that are affecting the profitability of our company.<br> Let's say we have the some financial data at our disposal. Let's load in the data, and take a look...

In [2]:
financial_data_path = 'data/NotRealCorp_financial_data.json'
financial_data = pd.read_json(financial_data_path)
financial_data.head(5)


,Year,Quarter,Distribution channel,Revenue ($M),Costs ($M),Customer count,Time
0,2021,Q1,Online Sales,1.50,1.301953,150,2021 Q1
1,2021,Q1,Direct Sales,1.50,1.380809,151,2021 Q1
2,2021,Q1,Retail Partners,1.50,1.348246,152,2021 Q1
3,2021,Q2,Online Sales,1.52,1.308608,152,2021 Q2
4,2021,Q2,Direct Sales,1.52,1.413305,153,2021 Q2


As you can see, this data has quarterly revenue, costs and customer data across different distribution channels. Let's create an Assistant
that can act as a personal analyst and make a nice visualization for our PowerPoint!

First, we need to upload our file so our Assistant can access it.

In [3]:
file = client.files.create(
  file=open('data/NotRealCorp_financial_data.json',"rb"),
  purpose='assistants',
)


Now, we're ready to create our Assistant. We can instruct our assistant to act as a data scientist, and take any queries we give it and run the necessary code to output the proper data visualization. We can also turn on the tool of Code Interpreter, so our Assistant will be able to code. Finally, we can specifiy any files we want to use, which in this case is just the `financial_data` file we created above.

In [4]:
assistant = client.beta.assistants.create(
  instructions="You are a data scientist assistant. When given data and a query, write the proper code and create the proper visualization",
  model="gpt-4-1106-preview",
  tools=[{"type": "code_interpreter"}],
  file_ids=[file.id]
)


Let's create a thread now, and as our first request ask the Assistant to calculate quarterly profits, and then plot the profits by distribution channel over time. The assistant will automatically calculate the profit for each quarter, and also create a new column combining quarter and year, without us having to ask for that directly. We can also specify the colors of each line.

In [5]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Calculate profit (revenue minus cost) by quarter and year, and visualize as a line plot across the distribution channels, where the colors of the lines are green, light red, and light blue",
      "file_ids": [file.id]
    }
  ]
)


No we can execute the run of our thread

In [6]:

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)


We can now start a loop that will check if the image has been created. Note: This may take a few minutes

In [7]:
import time

while True:
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    try:
        #See if image has been created
        messages.data[0].content[0].image_file
        print('Plot created!')
        break
    except:
        time.sleep(10)
        print('Assistant still working...')


Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Plot created!


Let's see the messages the Assistant added.

In [8]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
[message.content[0] for message in messages.data]


[MessageContentImageFile(image_file=ImageFile(file_id='file-UmzlAvR5fPEYizAvDAsdAeJB'), type='image_file'),
 MessageContentText(text=Text(annotations=[], value="The data has now been successfully loaded into a DataFrame. It contains the following columns: `Year`, `Quarter`, `Distribution channel`, `Revenue ($M)`, `Costs ($M)`, `Customer count`, and `Time`. To calculate profit by quarter and year, we need to compute the profit for each row as `Revenue ($M)` minus `Costs ($M)`, then group by `Year`, `Quarter`, and `Distribution channel` to aggregate the data.\n\nLet's proceed with these steps and then visualize the profit for each distribution channel as a line plot, with the lines colored green, light red, and light blue."), type='text'),
 MessageContentText(text=Text(annotations=[], value='The raw content of the file appears to be JSON, but the previous attempt to parse it failed because it seems that I directly tried to interpret the DataFrame column header as JSON, which was incorrec

We can see that the last message (latest message is shown first) from the assistant contains the image file we are looking for.  We can write a quick helper function to convert our output file to a png

In [9]:
def convert_file_to_png(file_id, write_path):
    data = client.files.content(file_id)
    data_bytes = data.read()
    with open(write_path, "wb") as file:
        file.write(data_bytes)


In [10]:
plot_file_id = messages.data[0].content[0].image_file.file_id
image_path = "data/NotRealCorp_chart.png"
convert_file_to_png(plot_file_id,image_path)


Let's load in the plot!

![Financial data plot](data/NotRealCorp_chart.png)

Nice! So, with just one sentence, we were able to have our assistant use code interpreter to
calculate the profitability, and graph the three lineplots of the various distribution channels.<br><br>
Now we have a nice visual for our slide, but we want some insights to go along with it. Let's use GPT-Vision to help us generate those.

## 2. Generating insights

To get insights from our image, we simply need to add a new message to our thread. Our Assistant will know to use GPT-Vision, and can give us some concise takeaways from the visual provided.

In [12]:
submit_message(assistant.id,thread,"Give me two medium length sentences (~20-30 words per sentence) of the \
      most important insights from the plot you just created.\
             These will be used for a slide deck, and they should be about the\
                     'so what' behind the data."
)


Run(id='run_kMFCS9y7f0EKXkVEVX0MJEAC', assistant_id='asst_7g0ZZvr42G9vxVBHyNCy2rIn', cancelled_at=None, completed_at=None, created_at=1701663813, expires_at=1701664413, failed_at=None, file_ids=['file-1JMqJZGth54cPsMKdwuCgmYG'], instructions='You are a data scientist assistant. When given data and a query, write the proper code and create the proper visualization', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_jpiHVuCY2Vr0wPXeLVk9zExB', tools=[ToolAssistantToolsCode(type='code_interpreter')])

Now, once the run has completed, we can get the latest message

In [13]:
# Hard coded wait for a response, as the assistant may iterate on the bullets.
time.sleep(10)
response = get_response(thread)
bullet_points = response.data[0].content[0].text.value
print(bullet_points)


The line plot indicates that "Online Sales" consistently outperformed other channels in profitability, showing a steady upward trend throughout the observed quarters. "Direct Sales" and "Retail Partners" exhibited more variability, with "Retail Partners" experiencing a notable profit increase in later periods.


Cool! Now we have some two insights to accompany our plot. Now let's get a compelling title for the slide.

In [14]:
submit_message(assistant.id,thread,"Given the plot and bullet points you created,\
 come up with a very brief title for a slide. It should reflect just the main insights you came up with."
)


Run(id='run_L64nABxuBXpu2KqgfT7yhUBA', assistant_id='asst_7g0ZZvr42G9vxVBHyNCy2rIn', cancelled_at=None, completed_at=None, created_at=1701663831, expires_at=1701664431, failed_at=None, file_ids=['file-1JMqJZGth54cPsMKdwuCgmYG'], instructions='You are a data scientist assistant. When given data and a query, write the proper code and create the proper visualization', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_jpiHVuCY2Vr0wPXeLVk9zExB', tools=[ToolAssistantToolsCode(type='code_interpreter')])

And the title is

In [15]:
#Wait as assistant may take a few steps
time.sleep(10)
response = get_response(thread)
title = response.data[0].content[0].text.value
print(title)


"Online Sales Lead Profit Growth Across Channels"


## 3. DALL-E3 title image

Nice, now we have a title, a plot and two bullet points. We're almost ready to put this all on a slide, but as a final step, let's have DALL-E3 come up with an image to use as the title slide of the presentation. <br><br>
*Note:* DALL-E3 is not yet available within the assistants API but is coming soon! <br> <br>
We'll feed in a brief description of our company (NotRealCorp) and have DALL-E3 do the rest!

In [16]:
company_summary = "NotReal Corp is a prominent hardware company that manufactures and sells processors, graphics cards and other essential computer hardware."


In [17]:
response = client.images.generate(
  model='dall-e-3',
  prompt=f"given this company summary {company_summary}, create an inspirational \
    photo showing the growth and path forward. This will be used at a quarterly\
       financial planning meeting",
       size="1024x1024",
       quality="hd",
       n=1
)
image_url = response.data[0].url


Let's take a look at our image

![Image](data/dalle_img.png)


Cool, now we can add this image to our thread, and ask the Assistant API to write the code to create our slide!

First, we can save the image locally, then upload it using the `File` upload endpoint.

In [18]:
dalle_img_path = 'data/dalle_image.png'
img = requests.get(image_url)

#Save locally
with open(dalle_img_path,'wb') as file:
  file.write(img.content)

#Upload
file = client.files.create(
  file=open(dalle_img_path, "rb"),
  purpose='assistants'
)


Now, we can simply ask the Assistant API to create a PowerPoint using the content it has helped us generate so far. We generated the title DALL-E3 image separately, so we will explicitly upload the file, but otherwise will ask the Assistant to use the relevant data to generate two slides in whatever format we want.

In [19]:
submit_message(assistant.id,thread,"Output a pptx slide deck with two slides as follows:\
               MAKE SURE the first slide is titled 'NotRealCorp: Quarterly financial planning meeting, Q3 2023' at the top of the screen, with large heading font size. The image included should be large but positioned BELOW the text. IMPORTANT: Do not have the title text and images overlap. ALL SLIDES should have the top third JUST for Titles. Make sure the image is not taking up the whole screen, so the reader can see the entire title. The second\
               slide should include the image of the line plot you generated earlier, this should be the only image file you have made from earlier in the conversation, which shows profits by distribution channel on the LHS of the screen, and include the insights you then provided on that plot on the RHS of the screen, in bullet form.\
              The insights should be under a larger, teal blue heading that says 'Key Insights:'. The image and the insights should CENTERED in the page. Make sure the code for the slides have a black background and white text, and are sleek and clean.\
               MAKE SURE the title of the second slide is the title you generated earlier, and is larger title text at the top of the screen, and that the second slide contains the image of the line plot you generated earlier.\
               MAKE SURE the slide titles are all in one row, one line, and take up the entire width of the page.\
               Check your work as you go, and make sure you are following all initial instructions as you move take each step.",
              file_ids=[file.id]
)


Run(id='run_kPvYu6oKwMVmwocMRDqTm5IA', assistant_id='asst_7g0ZZvr42G9vxVBHyNCy2rIn', cancelled_at=None, completed_at=None, created_at=1701663863, expires_at=1701664463, failed_at=None, file_ids=['file-1JMqJZGth54cPsMKdwuCgmYG'], instructions='You are a data scientist assistant. When given data and a query, write the proper code and create the proper visualization', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_jpiHVuCY2Vr0wPXeLVk9zExB', tools=[ToolAssistantToolsCode(type='code_interpreter')])

In [20]:
#May take 1-3 mins
while True:
    try:
        response = get_response(thread)
        pptx_id = response.data[0].content[0].text.annotations[0].file_path.file_id
        print("Successfully retrieved pptx_id:", pptx_id)  # Add this line
        break
    except Exception as e:  # Catch the exception and print it
        print("Assistant still working on PPTX...")
        time.sleep(10)


Assistant still working on PPTX...
Assistant still working on PPTX...
Assistant still working on PPTX...
Assistant still working on PPTX...
Assistant still working on PPTX...
Assistant still working on PPTX...
Assistant still working on PPTX...
Assistant still working on PPTX...
Successfully retrieved pptx_id: file-GRuk8av2xSXtRdk5cGX1vcDj


In [21]:
ppt_file= client.files.content(pptx_id)
file_obj = io.BytesIO(ppt_file.read())
with open("data/ppt_generated.pptx", "wb") as f:
    f.write(file_obj.getbuffer())


## 4. Creating the slides

Now, we have a PPTX file saved with all of our created content!. <br>
We don't have a `seed` parameter yet in the Assistants API, so the DALL-E3 image and wordings will be slightly different from what you see when you run this notebook, due to the non-deterministic quality of LLMs, but the outputs should be directionally the same.

Let's look at the screenshots of the .pptx we just created using JUST the assistants API and DALL-E3, we have the following:

![Title slide](data/title_slide.png)

And for the data slide, we have:

![Data slide](data/data_viz_slide.png)

Woo! While these slides could use some small formatting tweaks, we have made some great content, using just the Assistants API, GPT-4, GPT-Vision, and DALL-E3. Hopefully this helps make the slide creation process a bit easier, and provides a glimpse into how helpful the Assistants API can be!